<a href="https://colab.research.google.com/github/kesavakrishna/NLP-Resume-Ranking/blob/main/Resume_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("/content/UpdatedResumeDataSet.csv")
df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [3]:
df.shape

(962, 2)

In [4]:
df['Category'].value_counts()

Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Blockchain                   40
ETL Developer                40
Operations Manager           40
Data Science                 40
Sales                        40
Mechanical Engineer          40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64

In [5]:
df.Category.unique()

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [6]:
df["Resume"]

0      Skills * Programming Languages: Python (pandas...
1      Education Details \r\nMay 2013 to May 2017 B.E...
2      Areas of Interest Deep Learning, Control Syste...
3      Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4      Education Details \r\n MCA   YMCAUST,  Faridab...
                             ...                        
957    Computer Skills: â¢ Proficient in MS office (...
958    â Willingness to accept the challenges. â ...
959    PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960    COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961    Skill Set OS Windows XP/7/8/8.1/10 Database MY...
Name: Resume, Length: 962, dtype: object

cleaning data

In [7]:
import re

def cleanResume(txt):
  cleanText = re.sub('http\S+\s',' ', txt)
  cleanText = re.sub('RT|cc',' ', cleanText)
  cleanText = re.sub('#\S+\s',' ', cleanText)
  cleanText = re.sub('@\s+',' ', cleanText)
  cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
  cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
  cleanText = re.sub('\s+', ' ', cleanText)
  return cleanText

In [8]:
df['Resume'] = df['Resume'].apply(lambda x: cleanResume(x))

In [9]:
df['Resume']

0      Skills Programming Languages Python pandas num...
1      Education Details May 2013 to May 2017 B E UIT...
2      Areas of Interest Deep Learning Control System...
3      Skills R Python SAP HANA Tableau SAP HANA SQL ...
4      Education Details MCA YMCAUST Faridabad Haryan...
                             ...                        
957    Computer Skills Proficient in MS office Word B...
958     Willingness to a ept the challenges Positive ...
959    PERSONAL SKILLS Quick learner Eagerness to lea...
960    COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...
961    Skill Set OS Windows XP 7 8 8 1 10 Database MY...
Name: Resume, Length: 962, dtype: object

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

In [12]:
df


,Category,Resume
0,6,Skills Programming Languages Python pandas num...
1,6,Education Details May 2013 to May 2017 B E UIT...
2,6,Areas of Interest Deep Learning Control System...
3,6,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,6,Education Details MCA YMCAUST Faridabad Haryan...
...,...,...
957,23,Computer Skills Proficient in MS office Word B...
958,23,Willingness to a ept the challenges Positive ...
959,23,PERSONAL SKILLS Quick learner Eagerness to lea...
960,23,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...


In [13]:
df.Category.unique()

array([ 6, 12,  0,  1, 24, 16, 22, 14,  5, 15,  4, 21,  2, 11, 18, 20,  8,
       17, 19,  7, 13, 10,  9,  3, 23])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

tfidf.fit(df['Resume'])
requiredtext = tfidf.transform(df['Resume'])

In [15]:
requiredtext

<962x7352 sparse matrix of type '<class 'numpy.float64'>'
	with 164269 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(requiredtext, df['Category'], test_size=0.33, random_state=42)

In [18]:
X_test.shape

(318, 7352)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_pred)
print(accuracy_score(y_test, y_pred))

[15 15 15 13 14 17 16  2  0 14 13 12 16 23 20  5  6  4 10  9 19  1 10 23
 23 21 22 22  2 12 18  1  8 24 11 23  7 12 24  8 18  6  8 19 24 23 21  1
 15  4 15 22 11  5 15 13  1 19  5 12 22 22 20 24 21 18 12 10 10 20 10  8
  9 21 17 21  0 17 16 14 15 11 11  8 20  3 19  8  0  2  9 10  2 23 20 20
 23 12 18 12  7 16  8 14 18  3 14 19 14 14 15 18  8  2 21 18 23 10 23  5
 11 15 12  3  5  3  7 12 19  8 20 19  3 15  9 19  1 23 21  5 20 15 16  7
  7  8 15 18  1 15 13 20  7  4 18 11  5 15  5 12  9 22 18 21  8 23  4 12
 24 16 15 22  8 22  3 16 23 23 12  7 16 18  5  3 18  8 23 23 20 21  6  7
 23  2  3 18 14  1 12 13 22 12 11 23 18 15 19 15  6  0 15  8  9 16  6 12
 14  9 15  4  0 20 16  7  8 23  3 23  9  6  0  6  9 14 15 24  9  3  1 15
 13  5  6 12 11 15  8 21 16  4 12  8 21 20  5  9 22 13 16 19 15  4 22 10
 23 13  2 15 15  4 17 15  6 24 23 15  3 15 23 18 18 20 23 13 10 20 11 23
  4 22 15 23 16 12 24 17 20  5  8  5  1 18 20 11  6 17 18  5 23 21  8 24
  1 15 13  3  0 23]
0.9874213836477987


In [51]:
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision:", precision_score(y_test, y_pred, average='weighted')*100)
print("Recall:", recall_score(y_test, y_pred, average='weighted')*100)
print("F1 Score", f1_score(y_test, y_pred, average='weighted')*100)
print("Accuracy", accuracy_score(y_test, y_pred)*100)

Precision: 98.91334730957372
Recall: 98.74213836477988
F1 Score 98.7439394824464
Accuracy 98.74213836477988


In [23]:
import pickle
pickle.dump(tfidf,open('tfidf.pkl', 'wb'))
pickle.dump(clf,open('clf.pkl', 'wb'))

In [25]:
myresume = """Kesava Sai Krishna Kondepudi
(+91) 8008407440• kesavakrishna91@gmail.com • Linkedin • Github
EDUCATION
B.Tech. Electronics and Computer Engineering Oct 2020 – Present
Mahindra École Centrale, Hyderabad, India
Relevant Coursework: Machine Learning, Web Technologies, Object Oriented Programming, Software Engineering, Digital
System Design, Enterprise Risk Management(Level 1)
TECHNICAL SKILLS
Programming Languages: Python, C, C++, SQL, HTML, CSS, JavaScript, Java, Verilog HDL, Dart
Software: Google Colab, Intel Altera Quartus Prime, MATLAB, LTspice, Flutter, Arduino
Libraries: Pandas, Seaborn, Sklearn, Numpy, Tensorflow, Keras
Certifications: Python for Everybody(UMich/Coursera), Machine Learning(Stanford/Coursera), Introductory C
Programming(Duke/Coursera), Python and Data Visualization Bootcamps(Devtown/GDG/MSP), ERM(Level 1)
PROFESSIONAL EXPERIENCE
Feynn Labs, India: Machine Learning intern Jul 2023 - Sep 2023
● Developing and implementing machine learning models to analyze medium/large datasets related to consumer
behavior, market trends, and other relevant factors, including working on data cleaning and preprocessing,
model selection and tuning, and visualization of results.
Mathworks, AICTE, India: Artificial Intelligence Virtual Internship May 2023 - Sep 2023
● Gained hands-on experience with MATLAB and Simulink where I applied machine learning, deep learning, and
image processing to solve real-world problems
Coincent.ai, India: Full Stack Developer Sep 2022 - Oct 2022
● Full Stack Development internship at Coincent in collaboration with IBM, developing a Job portal website using
HTML, CSS, JavaScript, PHP, and MYSQL.
PUBLICATIONS
● A. Sahjwani, N. Choudhary, K. S. K. Kondepudi, A. Waghray and B. S. Vegi, "A Deep Learning Based Approach for
Cardiovascular Monitoring Using Seismocardiogram Signals," 2023 First International Conference on Cyber
Physical Systems, Power Electronics and Electric Vehicles (ICPEEV), Hyderabad, India, 2023, pp. 1-10, doi:
10.1109/ICPEEV58650.2023.10391941.
PROJECTS
Door2Destination, Mahindra University Feb 2023 - Jun 2023
● Designed and developed an app for outpass requests and ride-sharing bookings as a key component of a hostel
management project.
ASEAN-5 Stock Price Prediction, Mahindra University Feb 2023 - Jun 2023
● Predicted the ASEAN - 5 stock market index using various single classifier algorithms and ensemble algorithms.
TravelTrek, Mahindra University Apr 2023 - May 2023
● Designed a user-friendly travel website as part of a UI/UX project, focusing on enhancing the user experience and
interface design.
Black Pearl, AUVMEC, Mahindra University Apr 2022 - Sept 2022
● Programmed a software-hardware interface for an Underwater Remotely Operated Robot to work with XBOX
controllers.
● Worked with interfacing of an Autonomous Underwater Vehicle using Pixhawk and QGroundControl.
Heavy Machinery Sale Price Prediction, Mahindra University May 2022
● Predicted the sale price of heavy machinery using Machine Learning and Random Forests algorithm.
EXTRACURRICULAR EXPERIENCE
● Presented my paper “Forecasting ASEAN-5 Stock Index Price Movement Using Machine Learning Techniques” at
the 9th Pan IIM World Management Conference 2023 held in IIM Sambalpur.
● Placed 3rd in the Inquisitive Quiz, 4th in the MU Hacks Hackathon at Mahindra University (May 2022), and 6th in
the Sakshi K-Circle Quiz, Hyderabad, TG (Sept 2015).
"""

In [26]:
import pickle

# Load the trained classifier
clf = pickle.load(open('clf.pkl', 'rb'))
tfidfd = pickle.load(open('tfidf.pkl','rb'))

# Clean the input resume
cleaned_resume = cleanResume(myresume)

# Transform the cleaned resume using the trained TfidfVectorizer
input_features = tfidfd.transform([cleaned_resume])

# Make the prediction using the loaded classifier
prediction_id = clf.predict(input_features)[0]

# Map category ID to category name
category_mapping = {
    15: "Java Developer",
    23: "Testing",
    8: "DevOps Engineer",
    20: "Python Developer",
    24: "Web Designing",
    12: "HR",
    13: "Hadoop",
    3: "Blockchain",
    10: "ETL Developer",
    18: "Operations Manager",
    6: "Data Science",
    22: "Sales",
    16: "Mechanical Engineer",
    1: "Arts",
    7: "Database",
    11: "Electrical Engineering",
    14: "Health and fitness",
    19: "PMO",
    4: "Business Analyst",
    9: "DotNet Developer",
    2: "Automation Testing",
    17: "Network Security Engineer",
    21: "SAP Developer",
    5: "Civil Engineer",
    0: "Advocate",
}

category_name = category_mapping.get(prediction_id, "Unknown")

print("Predicted Category:", category_name)
print(prediction_id)

Predicted Category: Data Science
6
